# Algorithm Explained

In [2]:
# from utility.RetroParser import RetroData
from utility.RetroGateModel import RetroRLModel
from utility.RetroRLAgent import RetroRLAgent
# from utility.ResultProcess import ResultParser
from utility.DataPrepare import Prepare
from utility.BruteForceSearch import expansion, Product, Reaction
import time
import numpy as np
# # Use Braket SDK Cost Tracking to estimate the cost to run this example
# from braket.tracking import Tracker
# t = Tracker().start()

timestamp = time.strftime("%Y%m%d-%H")

ModuleNotFoundError: No module named 'utility.RetroRLAgent'

In [2]:
# config your aws account in your ~/.aws/config
import os
os.environ['AWS_DEFAULT_REGION']='us-west-1'

#### Step 1: Prepare Data

In this part, we load the retrosynthesis prediction data for experiment.
The [USPTO-50K](https://tdcommons.ai/generation_tasks/retrosyn/#uspto-50k) was 
put in the repository. We assign the relative 
path to **raw_path**.
The **s3_bucket** and **prefix** are used to store the 
results. We can use the one created with the 
cloudformation for convenience.

In [3]:
data_path = 'data'
# download dateset
!mkdir $data_path
!mkdir $data_path\smiles
!wget https://d1o8djwwk7diqy.cloudfront.net/retrosynthetic-plannin-dataset.zip
!unzip retrosynthetic-plannin-dataset.zip
# windows
!copy retrosynthetic-planning-dataset $data_path
!copy data\smiles_map.npy  data\smiles\smiles_map.npy

# linux
# !cp retrosynthetic-planning-dataset $data_path
# !cp data/smiles_map.npy  data/smiles

--2023-04-26 14:09:50--  https://d1o8djwwk7diqy.cloudfront.net/retrosynthetic-plannin-dataset.zip
Resolving d1o8djwwk7diqy.cloudfront.net (d1o8djwwk7diqy.cloudfront.net)... 18.155.70.118, 18.155.70.191, 18.155.70.129, ...
Connecting to d1o8djwwk7diqy.cloudfront.net (d1o8djwwk7diqy.cloudfront.net)|18.155.70.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3841896 (3.7M) [application/zip]
Saving to: 'retrosynthetic-plannin-dataset.zip'

     0K .......... .......... .......... .......... ..........  1%  630K 6s
    50K .......... .......... .......... .......... ..........  2%  864K 5s
   100K .......... .......... .......... .......... ..........  3% 2.58M 4s
   150K .......... .......... .......... .......... ..........  5% 1.20M 3s
   200K .......... .......... .......... .......... ..........  6% 2.05M 3s
   250K .......... .......... .......... .......... ..........  7% 3.04M 3s
   300K .......... .......... .......... .......... ..........  9% 1.09M 3s


Archive:  retrosynthetic-plannin-dataset.zip
   creating: retrosynthetic-planning-dataset/
  inflating: retrosynthetic-planning-dataset/buyable.npy  
  inflating: retrosynthetic-planning-dataset/target_product.npy  
  inflating: retrosynthetic-planning-dataset/reactions_dictionary.npy  
  inflating: retrosynthetic-planning-dataset/smiles_map.npy  
  inflating: retrosynthetic-planning-dataset/ground_truth.npy  
  inflating: retrosynthetic-planning-dataset/Deadend.npy  
  inflating: retrosynthetic-planning-dataset/uspto50k.xlsx  
  inflating: retrosynthetic-planning-dataset/smiles_dictionary.npy  
retrosynthetic-planning-dataset\buyable.npy
retrosynthetic-planning-dataset\Deadend.npy
retrosynthetic-planning-dataset\ground_truth.npy
retrosynthetic-planning-dataset\reactions_dictionary.npy
retrosynthetic-planning-dataset\smiles_dictionary.npy
retrosynthetic-planning-dataset\smiles_map.npy
retrosynthetic-planning-dataset\target_product.npy
retrosynthetic-planning-dataset\uspto50k.xlsx
已复制  

In [3]:
# input: predata_uspto-50k.xlsx
# output: file1.npy,file2.npy
raw_path = 'data/uspto50k.xlsx'
prepare = Prepare(raw_path)
prepare.generate_files()  # 
prepare.generate_ground_truth()
ground_truth = np.load(prepare.path+'ground_truth.npy', allow_pickle=True).tolist()

INFO:root:Files are present.
INFO:root:All files are generated!
INFO:root:File is present.


#### Step 2: Build Model

In this part, we build the circuit model for retrosynthetic planning

In [32]:
# initial the RetroRLModel object
init_param = {}
method = ['retro-rl', 'retro-qrl']

for mt in method:
    if mt == 'retro-rl':
        init_param[mt] = {}
        init_param[mt]['param'] = ['inputsize', 'middlesize', 'outputsize']
    elif mt == 'retro-qrl':
        init_param[mt] = {}
        init_param[mt]['param'] = ['n_qubits', 'device', 'framework', 'shots', 'layers']
    
retro_rl_model = RetroRLModel(data=None, method=method, **init_param)

INFO:root:initial reinforcement learning for retrosynthetic-planning
INFO:root:initial quantum reinforcement learning for retrosynthetic-planning


In [33]:
model_param={}
method = 'retro-rl'
model_param[method] = {}
model_param[method]['inputsize'] = [256]
model_param[method]['middlesize'] = [256,512,1024]
model_param[method]['outputsize'] = [1]

retro_rl_model.build_model(**model_param)

INFO:root:Construct model for inputsize:256,middlesize:256,outputsize:1 0.0 min
INFO:root:Construct model for inputsize:256,middlesize:512,outputsize:1 1.665353775024414e-05 min
INFO:root:Construct model for inputsize:256,middlesize:1024,outputsize:1 1.6490618387858074e-05 min


In [34]:
model_param={}
method = 'retro-qrl'
model_param[method] = {}
model_param[method]['n_qubits'] = [8]
model_param[method]['device'] = ['local', 'sv1', 'aspen-m2']
model_param[method]['framework'] = ['pennylane']
model_param[method]['shots'] = [100,1000]
model_param[method]['layers'] = [1,2,3]

retro_rl_model.build_model(**model_param)


INFO:root:Construct model for n_qubits:8,device:local,framework:pennylane,layers:1 1.6649564107259113e-05 min
INFO:root:Construct model for n_qubits:8,device:local,framework:pennylane,layers:2 1.6673405965169272e-05 min
INFO:root:Construct model for n_qubits:8,device:local,framework:pennylane,layers:3 0.0 min
INFO:root:Construct model for n_qubits:8,device:local,framework:pennylane,layers:1 0.0 min
INFO:root:Construct model for n_qubits:8,device:local,framework:pennylane,layers:2 0.0 min
INFO:root:Construct model for n_qubits:8,device:local,framework:pennylane,layers:3 0.0 min
INFO:root:Construct model for n_qubits:8,device:sv1,framework:pennylane,layers:1 1.6756852467854817e-05 min
INFO:root:Construct model for n_qubits:8,device:sv1,framework:pennylane,layers:2 0.0 min
INFO:root:Construct model for n_qubits:8,device:sv1,framework:pennylane,layers:3 0.0 min
INFO:root:Construct model for n_qubits:8,device:sv1,framework:pennylane,layers:1 0.0 min
INFO:root:Construct model for n_qubits:8,

We can use the following method to check the properties of 
model. This way, we can build many models conveniently. 
After that, we save the model and update the value of 
**model_path**.

In [35]:
# describe the model parameters
model_info = retro_rl_model.describe_model()

INFO:root:method: retro-rl
INFO:root:param: inputsize, value {256}
INFO:root:param: middlesize, value {256, 512, 1024}
INFO:root:param: outputsize, value {1}
INFO:root:method: retro-qrl
INFO:root:param: n_qubits, value {8}
INFO:root:param: device, value {'sv1', 'aspen-m2', 'local'}
INFO:root:param: framework, value {'pennylane'}
INFO:root:param: shots, value {1000, 100}
INFO:root:param: layers, value {1, 2, 3}


In [36]:
# save the model
model_path = retro_rl_model.save("latest")

print(f"You have built the nn model for RL and saved it as {model_path}")

INFO:root:finish save retrorl_model_latest.pickle


You have built the nn model for RL and saved it as .\retrorl_model_latest.pickle


In [37]:
# !cp $model_path $data_path

# windows
!copy $model_path $data_path

已复制         1 个文件。


#### Step 3: Learn Retrosynthetic Planning

In this part, we use cpu to run classical model for retrosynthetic planning 
and simulators/NISQ devices to run quantum model for retrosysnthetic planning.

In [10]:
model_path='./retrorl_model_latest.pickle'

# get the model you want to optimize
n_qubits = 8
device = 'sv1'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

In [11]:
# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "hybrid-job"

In [12]:
data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

job_arn = retro_rl_agent.get_job_arn()
print(f"create job with arn {job_arn}")

INFO:root:load data...
INFO:root:model is None
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Going to run sv1 mode
create job with arn arn:aws:braket:us-west-1:493904798517:job/retrorl-job-sv1-torch-1682489607


In [13]:
retro_rl_agent.get_job().cancel()

'CANCELLING'

In [14]:
# get the model you want to optimize
n_qubits = 8
device = 'local'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

job_arn = retro_rl_agent.get_job_arn()
print(f"create job with arn {job_arn}")

INFO:root:load data...
INFO:root:model is None
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Going to run local mode
create job with arn arn:aws:braket:us-west-1:493904798517:job/retrorl-job-local-torch-1682400869


In [16]:
retro_rl_agent.get_job().cancel()

'CANCELLING'

In [ ]:
# get the model you want to optimize
# config your aws account in your ~/.aws/config
import os
os.environ['AWS_DEFAULT_REGION']='us-west-1'

n_qubits = 8
device = 'aspen-m2'
framework = 'pennylane'
shots = 100

model_name = "{}_{}_{}_{}".format(n_qubits, device, framework, shots)
method = "retro-qrl"

data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

job_arn = retro_rl_agent.get_job_arn()
print(f"create job with arn {job_arn}")

In [ ]:
retro_rl_agent.get_job().cancel()

In [ ]:
model_path='./retrorl_model_latest.pickle'

# get the model you want to optimize
inputsize = 256
middlesize = 512
outputsize = 1

model_name = "{}_{}_{}".format(inputsize, middlesize, outputsize)
method = "retro-rl"

# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "hybrid-job"

In [ ]:
data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_rl_model = RetroRLModel.load(model_path)
model_info = retro_rl_model.describe_model()
retro_model = retro_rl_model.get_model(method, model_name)
retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game()
retro_rl_agent.save("latest", path='data')

In [ ]:
NN_path = retro_rl_agent.save("latest")

#### Step 4: PostProcess Result

In [11]:
# get the model you want to optimize
n_qubits = 8
device = 'local'
framework = 'pennylane'
shots = 100
layers = 2

model_name = "{}_{}_{}_{}_{}".format(n_qubits, device, framework, shots, layers)
method = "retro-qrl"

# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "local-instance"

data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_model = None
if train_mode == "local-instance":
    # get model
    retro_rl_model = RetroRLModel.load(model_path)
    model_info = retro_rl_model.describe_model()
    retro_model = retro_rl_model.get_model(method, model_name)

retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()


INFO:root:method: retro-rl
INFO:root:param: inputsize, value {256}
INFO:root:param: middlesize, value {512}
INFO:root:param: outputsize, value {1}
INFO:root:method: retro-qrl
INFO:root:param: n_qubits, value {8}
INFO:root:param: device, value {'sv1', 'aspen-m2', 'local'}
INFO:root:param: framework, value {'pennylane'}
INFO:root:param: shots, value {100}
INFO:root:param: layers, value {1, 2, 3}
INFO:root:load data...
INFO:root:model is {'model_name': '8_local_pennylane_100_2', 'version': '1682575107', 'nn_model': CirModel()}


('episode', 1)
('episode', 2)
('episode', 3)
('episode', 4)
('episode', 5)
('episode', 6)
('episode', 7)
('episode', 8)
('episode', 9)
('episode', 10)
('episode', 11)
('episode', 12)
('episode', 13)
('episode', 14)
('episode', 15)
('episode', 16)
('episode', 17)
('episode', 18)
('episode', 19)
('episode', 20)
epsiode 20 training...
Parameter containing:
tensor([[5.6262, 0.4769, 4.2342, 4.5672, 1.8381, 3.4660, 4.2988, 2.5562],
        [4.5192, 0.0290, 2.0298, 2.7523, 2.2296, 6.2124, 0.8331, 4.7189]],
       dtype=torch.float64, requires_grad=True)


KeyboardInterrupt: 

In [33]:
# get the model you want to optimize
inputsize = 256
middlesize = 512
outputsize = 1

model_name = "{}_{}_{}".format(inputsize, middlesize, outputsize)
method = "retro-rl"

# train_mode can be: "local-instance", "local-job", "hybrid-job"
train_mode = "local-instance"
data_path = 'data'
agent_param = {}
agent_param["data_path"] = data_path
agent_param["train_mode"] = train_mode
agent_param["model_name"] = model_name
agent_param["model_path"] = model_path

retro_rl_model = RetroRLModel.load(model_path)
model_info = retro_rl_model.describe_model()
retro_model = retro_rl_model.get_model(method, model_name)
retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
retro_rl_agent.game_job()

INFO:root:method: retro-rl
INFO:root:param: inputsize, value {256}
INFO:root:param: middlesize, value {512}
INFO:root:param: outputsize, value {1}
INFO:root:method: retro-qrl
INFO:root:param: n_qubits, value {8}
INFO:root:param: device, value {'aspen-m2', 'local', 'sv1'}
INFO:root:param: framework, value {'pennylane'}
INFO:root:param: shots, value {100}
INFO:root:param: layers, value {1, 2, 3}
INFO:root:load data...
INFO:root:model is {'model_name': '256_512_1', 'version': '1682491773', 'nn_model': Model(
  (relu): ReLU()
  (value_fc1): Linear(in_features=256, out_features=512, bias=True)
  (value_fc2): Linear(in_features=512, out_features=1, bias=True)
)}


('episode', 1)
('episode', 2)
('episode', 3)
('episode', 4)
('episode', 5)
('episode', 6)
('episode', 7)
('episode', 8)
('episode', 9)
('episode', 10)
('episode', 11)
('episode', 12)
('episode', 13)
('episode', 14)
('episode', 15)
('episode', 16)
('episode', 17)
('episode', 18)
('episode', 19)
('episode', 20)
epsiode 20 training...
finish epoch 0 for 4.9996376037597654e-05 minutes
finish epoch 1 for 3.337860107421875e-05 minutes
finish epoch 2 for 1.665353775024414e-05 minutes
finish epoch 3 for 1.6661485036214194e-05 minutes
finish epoch 4 for 1.6657511393229165e-05 minutes
finish epoch 5 for 1.668532689412435e-05 minutes
finish epoch 6 for 1.6657511393229165e-05 minutes
finish epoch 7 for 1.6673405965169272e-05 minutes
finish epoch 8 for 1.6657511393229165e-05 minutes
finish epoch 9 for 1.6669432322184246e-05 minutes
finish epoch 10 for 1.666545867919922e-05 minutes
finish epoch 11 for 1.6669432322184246e-05 minutes
finish epoch 12 for 1.6657511393229165e-05 minutes
finish epoch 13 f

In [18]:
target = 'COC(Cc1ccc2oc(Cc3nc(-c4ccccc4)oc3C)cc2c1)OC'
retro_rl_agent.pathway(target)
layer2 = retro_rl_agent.layer2
# print(layer2)
path = set()
for i, j in layer2.items():
    for k, l  in j.items():
        path.add(l)
print(f"The path of retro_rl_agent: \n \
 {path}")

input_data_path = agent_param["data_path"]
ground_truth = np.load(input_data_path+'/ground_truth.npy', allow_pickle=True).tolist()

real_path = set(ground_truth[target]['path'])
print(f"The real_path of Brute force: \n \
 {real_path}")
real_cost = ground_truth[target]['cost']
print(f"The real_cost of Brute force: \n \
 {real_cost}")
print(f"Get the same path: {path == real_path}")

The path of retro_rl_agent: 
  {'Cc1ccc(S(=O)(=O)O)cc1', 'COC(Cc1ccc2oc(Cc3nc(-c4ccccc4)oc3C)cc2c1)OC', 'O=C(O)[C@@H]1CCCN1', 'COC=Cc1ccc2oc(Cc3nc(-c4ccccc4)oc3C)cc2c1', 'O=[N+]([O-])c1ccc(CO)cc1'}
The real_path of Brute force: 
  {'Cc1ccc(S(=O)(=O)O)cc1', 'COC(Cc1ccc2oc(Cc3nc(-c4ccccc4)oc3C)cc2c1)OC', 'O=C(O)[C@@H]1CCCN1', 'COC=Cc1ccc2oc(Cc3nc(-c4ccccc4)oc3C)cc2c1', 'O=[N+]([O-])c1ccc(CO)cc1'}
The real_cost of Brute force: 
  102.0
Get the same path: True


In [19]:
target = 'O=C(NCc1ccc(CO)cc1)c1ccccn1'
retro_rl_agent.pathway(target)
layer2 = retro_rl_agent.layer2
# print(layer2)
path = set()
for i, j in layer2.items():
    for k, l  in j.items():
        path.add(l)
print(f"The path of retro_rl_agent: \n \
 {path}")

input_data_path = agent_param["data_path"]
ground_truth = np.load(input_data_path+'/ground_truth.npy', allow_pickle=True).tolist()

real_path = set(ground_truth[target]['path'])
print(f"The real_path of Brute force: \n \
 {real_path}")
real_cost = ground_truth[target]['cost']
print(f"The real_cost of Brute force: \n \
 {real_cost}")
print(f"Get the same path: {path == real_path}")

The path of retro_rl_agent: 
  {'O=C(NCc1ccc(CO)cc1)c1ccccn1', 'N#Cc1ccc(CO)cc1', 'O=C(O)c1ccccn1', 'NCc1ccc(CO)cc1', 'N#Cc1ccc(C=O)cc1'}
The real_path of Brute force: 
  {'O=C(NCc1ccc(CO)cc1)c1ccccn1', 'N#Cc1ccc(CO)cc1', 'O=C(O)c1ccccn1', 'NCc1ccc(CO)cc1', 'N#Cc1ccc(C=O)cc1'}
The real_cost of Brute force: 
  3.0
Get the same path: True


In [20]:
target = 'CCCCC(CC)COC(=O)C(C#N)=C(c1ccccc1)c1ccccc1'
retro_rl_agent.pathway(target)
layer2 = retro_rl_agent.layer2
# print(layer2)
path = set()
for i, j in layer2.items():
    for k, l  in j.items():
        path.add(l)
print(f"The path of retro_rl_agent: \n \
 {path}")

input_data_path = agent_param["data_path"]
ground_truth = np.load(input_data_path+'/ground_truth.npy', allow_pickle=True).tolist()

real_path = set(ground_truth[target]['path'])
print(f"The real_path of Brute force: \n \
 {real_path}")
real_cost = ground_truth[target]['cost']
print(f"The real_cost of Brute force: \n \
 {real_cost}")
print(f"Get the same path: {path == real_path}")

The path of retro_rl_agent: 
  {'O=C(c1ccccc1)c1ccccc1', 'c1ccccc1', 'CCCCC(CC)COC(=O)CC#N', 'O=C(Cl)c1ccccc1', 'CCCCC(CC)COC(=O)C(C#N)=C(c1ccccc1)c1ccccc1'}
The real_path of Brute force: 
  {'O=C(c1ccccc1)c1ccccc1', 'c1ccccc1', 'CCCCC(CC)COC(=O)CC#N', 'O=C(Cl)c1ccccc1', 'CCCCC(CC)COC(=O)C(C#N)=C(c1ccccc1)c1ccccc1'}
The real_cost of Brute force: 
  2.0
Get the same path: True


# Hybrid Job Experiment

In [ ]:
from braket.aws import AwsQuantumJob
from braket.jobs.config import InstanceConfig
import boto3
import json
import time
import altair as alt
import pandas as pd
import numpy as np
from utility.HybridJobHelpers import *

#### Step 1: Prepare parameters for batch evaluation

In this part, we set the parameters for batch evaluation

In [ ]:
# parameters for experiments
experiment_name = "retrosynthetic-planning"
data_path = "retrosynthetic-planning-data"
suffix_check = ["txt"]
experiments_params =  {
    "version": "1",
    "params": [
        {"n_qubits": [8]},
        {"framework": ['pennylane']},
        {"layers": [1,2,3]},
        {"shots": [100]},
        {"device": ['local']}
    ]
}

hp = {}
hybrid_job_params = []
parse_params(experiments_params['params'], hp, hybrid_job_params)

print(f"parameters for experiments: \n {hybrid_job_params}")

In [ ]:
# model_param={}
# method = 'retro-qrl'
# model_param[method] = {}
# model_param[method]['n_qubits'] = [8]
# model_param[method]['device'] = ['local', 'sv1', 'aspen-m2']
# model_param[method]['framework'] = ['pennylane']
# model_param[method]['shots'] = [100]
# model_param[method]['layers'] = [1,2,3]


# # get the model you want to optimize
# n_qubits = 8
# device = 'local'
# framework = 'pennylane'
# shots = 100
# layers = 2

# model_name = "{}_{}_{}_{}_{}".format(n_qubits, device, framework, shots, layers)
# method = "retro-qrl"

# # train_mode can be: "local-instance", "local-job", "hybrid-job"
# train_mode = "local-instance"

# data_path = 'data'
# agent_param = {}
# agent_param["data_path"] = data_path
# agent_param["train_mode"] = train_mode
# agent_param["model_name"] = model_name
# agent_param["model_path"] = model_path

# retro_model = None
# if train_mode == "local-instance":
#     # get model
#     retro_rl_model = RetroRLModel.load(model_path)
#     model_info = retro_rl_model.describe_model()
#     retro_model = retro_rl_model.get_model(method, model_name)

# retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
# retro_rl_agent.game_job()

In [ ]:
# model_param={}
# method = 'retro-rl'
# model_param[method] = {}
# model_param[method]['inputsize'] = [256]
# model_param[method]['middlesize'] = [512]
# model_param[method]['outputsize'] = [1]


# # get the model you want to optimize
# inputsize = 256
# middlesize = 512
# outputsize = 1

# model_name = "{}_{}_{}".format(inputsize, middlesize, outputsize)
# method = "retro-rl"

# # train_mode can be: "local-instance", "local-job", "hybrid-job"
# train_mode = "local-instance"
# data_path = 'data'
# agent_param = {}
# agent_param["data_path"] = data_path
# agent_param["train_mode"] = train_mode
# agent_param["model_name"] = model_name
# agent_param["model_path"] = model_path

# retro_rl_model = RetroRLModel.load(model_path)
# model_info = retro_rl_model.describe_model()
# retro_model = retro_rl_model.get_model(method, model_name)
# retro_rl_agent1 = RetroRLAgent(retro_model, method, **agent_param)
# retro_rl_agent1.game_job()

In [38]:
# parameters for experiments
experiment_name = "retrosynthetic-planning"
data_path = "retrosynthetic-planning-data"
suffix_check = ["txt"]
experiments_params =  {
    "version": "1",
    "params1": [
        {"n_qubits": [8]},
        {"framework": ['pennylane']},
        {"layers": [1,2,3]},
        {"shots": [100,1000]},
        {"device": ['local']}
    ],
    "params2": [
        {"inputsize": [256]},
        {"middlesize": [256,512,1024]},
        {"outputsize": [1]}
    ]
}

# hp = {}
# hybrid_job_params = []
# parse_params(experiments_params['params'], hp, hybrid_job_params)

# print(f"parameters for experiments: \n {hybrid_job_params}")

hybrid_job_params = []
for n_qubits in experiments_params['params1'][0]["n_qubits"]:
    for framework in experiments_params['params1'][1]["framework"]:
            for layers in experiments_params['params1'][2]["layers"]:
                for shots in experiments_params['params1'][3]["shots"]:
                    for device in experiments_params['params1'][4]["device"]:
                        model_name = "{}_{}_{}_{}_{}".format(n_qubits, device, framework, shots, layers)
                        hybrid_job_params.append(model_name)
for inputsize in experiments_params['params2'][0]["inputsize"]:
    for middlesize in experiments_params['params2'][1]["middlesize"]:
            for outputsize in experiments_params['params2'][2]["outputsize"]:                
                model_name = "{}_{}_{}".format(inputsize, middlesize, outputsize)
                hybrid_job_params.append(model_name)

In [39]:
hybrid_job_params

['8_local_pennylane_100_1',
 '8_local_pennylane_1000_1',
 '8_local_pennylane_100_2',
 '8_local_pennylane_1000_2',
 '8_local_pennylane_100_3',
 '8_local_pennylane_1000_3',
 '256_256_1',
 '256_512_1',
 '256_1024_1']

In [ ]:
avtocost = []
for i in hybrid_job_params:
    model_name = hybrid_job_params[i]
    if model_name[0] == "8":
        method = "retro-qrl"
    else:
        method = "retro-rl"
    

    # train_mode can be: "local-instance", "local-job", "hybrid-job"
    train_mode = "local-instance"

    data_path = 'data'
    agent_param = {}
    agent_param["data_path"] = data_path
    agent_param["train_mode"] = train_mode
    agent_param["model_name"] = model_name
    agent_param["model_path"] = model_path

    retro_model = None
    if train_mode == "local-instance":
        # get model
        retro_rl_model = RetroRLModel.load(model_path)
        model_info = retro_rl_model.describe_model()
        retro_model = retro_rl_model.get_model(method, model_name)

    retro_rl_agent = RetroRLAgent(retro_model, method, **agent_param)
    retro_rl_agent.game_job()
    avtocost.append(retro_rl_agent.avtocost)

In [ ]:
# draw the training curve
import matplotlib.pyplot as plt
plt.figure(1)
plt.title('Training curve')
plt.xlabel('Epoch')
plt.ylabel('Average cost')
for i in range(len(hybrid_job_params)):
    plt.plot(range(0,len(avtocost[0])),avtocost[i],label=hybrid_job_params[i])
plt.legend(loc = 'upper right')